In [1]:
# PRÉ-PROCESSAMENTO DOS DADOS

import pandas as pd
import numpy as np

# Caminho do arquivo
caminho_arquivo = "/content/drive/MyDrive/Colab Notebooks/Dados_climaticos_estação_mirante_SP.csv"
df = pd.read_csv(caminho_arquivo, sep=';')

# Conversão coluna de datas
if 'Data_Medicao' in df.columns:
    df['Data_Medicao'] = df['Data_Medicao'].str.strip()
    df['Data_Medicao'] = pd.to_datetime(df['Data_Medicao'], format='%d/%m/%Y', errors='coerce')

# Remover coluna ponto de orvalho
coluna_remover = 'Temperatura_ponto_orvalho_media_diario(AUT)(°C)'
if coluna_remover in df.columns:
    df.drop(columns=[coluna_remover], inplace=True)

# Substituir vírgulas por pontos e converter para float
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Preencher valores ausentes
coluna_chuva = 'Precipitacao_total_diario(AUT)(mm)'
if coluna_chuva in df.columns:
    df[coluna_chuva] = df[coluna_chuva].fillna(df[coluna_chuva].mean())

colunas_mediana = [
    'Pressao_atmosferica_media_diario(AUT)(mB)',
    'Temperatura_maxima_diario(AUT)(°C)',
    'Temperatura_minima_diario(AUT)(°C)',
    'Umidade_relativa_do_ar_media_diario(AUT)(%)',
    'Vento_velocidade_media_diario(AUT)(m/s)'
]

for col in colunas_mediana:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

df['Umidade_relativa_do_ar_media_diario(AUT)(%)'] = df['Umidade_relativa_do_ar_media_diario(AUT)(%)'].round(0).astype(int)



In [2]:
# ENGENHARIA DE ATRIBUTOS

# Chuva acumulada últimos 3 dias
df['Chuva_3dias(mm)'] = df['Precipitacao_total_diario(AUT)(mm)'].rolling(window=3).sum()

# Temperatura média últimos 3 dias
df['Temp_media_3dias(°C)'] = df[['Temperatura_maxima_diario(AUT)(°C)', 'Temperatura_minima_diario(AUT)(°C)']].mean(axis=1).rolling(window=3).mean()

# Variação diária de temperatura
df['Variacao_Temp(°C)'] = df['Temperatura_maxima_diario(AUT)(°C)'] - df['Temperatura_minima_diario(AUT)(°C)']

# Variações absolutas de pressão e umidade em relação ao dia anterior
df['Variacao_Pressao(mB)'] = df['Pressao_atmosferica_media_diario(AUT)(mB)'].diff().abs()
df['Variacao_Umidade(%)'] = df['Umidade_relativa_do_ar_media_diario(AUT)(%)'].diff().abs()

# Preencher valor ausente com média
colunas_auxiliares = [
    'Chuva_3dias(mm)',
    'Temp_media_3dias(°C)',
    'Variacao_Pressao(mB)',
    'Variacao_Umidade(%)'
]
df[colunas_auxiliares] = df[colunas_auxiliares].apply(lambda x: x.fillna(x.mean()))

# Arredonda todas as novas colunas para 1 casa decimal
colunas_para_arredondar = colunas_auxiliares + ['Variacao_Temp(°C)']
df[colunas_para_arredondar] = df[colunas_para_arredondar].round(1)


In [3]:
# CÁLCULO DE PROBABILIDADE DE DESASTRE

def detectar_probabilidade_desastre(row):
    LIMIARES = {
        "chuva_dia": 80,
        "chuva_3dias": 120,
        "vento": 20,
        "umidade": 90,
        "onda_calor": 33
    }

    PESOS = {
        "chuva_dia": 0.5,
        "chuva_3dias": 0.3,
        "vento": 0.1,
        "umidade": 0.05,
        "onda_calor": 0.05
    }

    score = sum([
        PESOS["chuva_dia"]   if row['Precipitacao_total_diario(AUT)(mm)'] >= LIMIARES["chuva_dia"] else 0,
        PESOS["chuva_3dias"] if row['Chuva_3dias(mm)'] >= LIMIARES["chuva_3dias"] else 0,
        PESOS["vento"]        if row['Vento_velocidade_media_diario(AUT)(m/s)'] >= LIMIARES["vento"] else 0,
        PESOS["umidade"]      if row['Umidade_relativa_do_ar_media_diario(AUT)(%)'] >= LIMIARES["umidade"] else 0,
        PESOS["onda_calor"]   if row['Temp_media_3dias(°C)'] >= LIMIARES["onda_calor"] else 0
    ])

    score = min(score, 1)
    return round(score, 2)

# Aplicando a função ao df
df['Probabilidade_Desastre(%)'] = df.apply(detectar_probabilidade_desastre, axis=1)

# Convertendo para porcentagem
df['Probabilidade_Desastre(%)'] = (df['Probabilidade_Desastre(%)'] * 100).round().astype(int)


In [4]:
# TREINAMENTO E AVALIAÇÃO E DO MODELO

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Lista de variáveis independentes (features)
features = [
    'Precipitacao_total_diario(AUT)(mm)',
    'Pressao_atmosferica_media_diario(AUT)(mB)',
    'Temperatura_maxima_diario(AUT)(°C)',
    'Temperatura_minima_diario(AUT)(°C)',
    'Umidade_relativa_do_ar_media_diario(AUT)(%)',
    'Vento_velocidade_media_diario(AUT)(m/s)',
    'Chuva_3dias(mm)',
    'Temp_media_3dias(°C)',
    'Variacao_Temp(°C)',
    'Variacao_Pressao(mB)',
    'Variacao_Umidade(%)'
]

# Variável alvo
target = 'Probabilidade_Desastre(%)'

# Separação entre variáveis independentes (X) e variável alvo (y)
X = df[features]
y = df[target]

# Divisão em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo de regressão
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Previsão no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação do modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Resultados
print("\U0001F4BB Treinamento e Avaliação do Modelo (Random Forest Regressor)")
print(f"Erro Quadrático Médio (MSE): {mse:.2f}")
print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")


# Salvar o modelo treinado
joblib.dump(model, 'modelo_random_forest.pkl')

# Salvar a lista de features usadas
joblib.dump(features, 'lista_features.pkl')


💻 Treinamento e Avaliação do Modelo (Random Forest Regressor)
Erro Quadrático Médio (MSE): 3.69
Erro Absoluto Médio (MAE): 0.11
Coeficiente de Determinação (R²): 0.85


['lista_features.pkl']

In [5]:
# IMPORTAÇÃO DE NOVOS DADOS (VIA API OPEN-METEO)

import requests
import pandas as pd
from datetime import datetime, timedelta

# Coordenadas de São Paulo
latitude = -23.5505
longitude = -46.6333

# Intervalo da previsão (hoje até 6 dias à frente)
data_inicio = datetime.today().strftime('%Y-%m-%d')
data_fim = (datetime.today() + timedelta(days=6)).strftime('%Y-%m-%d')

# Variáveis diárias e horárias
variaveis_diarias = [
    "temperature_2m_max",
    "temperature_2m_min",
    "precipitation_sum",
    "windspeed_10m_max",
    "relative_humidity_2m_mean"
]
variaveis_horarias = ["surface_pressure"]

# Requisição da API
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}" \
      f"&daily={','.join(variaveis_diarias)}" \
      f"&hourly={','.join(variaveis_horarias)}" \
      f"&timezone=America%2FSao_Paulo&start_date={data_inicio}&end_date={data_fim}"

response = requests.get(url)
data = response.json()

# Verifica se os dados vieram corretamente
if "daily" not in data or "hourly" not in data:
    print("Erro na resposta da API:")
    print(data)
else:
    # Dados diários
    df_daily = pd.DataFrame({
        "Data_Medicao": data["daily"]["time"],
        "Temperatura_maxima_diario(AUT)(°C)": data["daily"]["temperature_2m_max"],
        "Temperatura_minima_diario(AUT)(°C)": data["daily"]["temperature_2m_min"],
        "Precipitacao_total_diario(AUT)(mm)": data["daily"]["precipitation_sum"],
        "Umidade_relativa_do_ar_media_diario(AUT)(%)": data["daily"]["relative_humidity_2m_mean"],
        "Vento_velocidade_media_diario(AUT)(m/s)": data["daily"]["windspeed_10m_max"]
    })

    # Processa pressão horária
    df_hourly = pd.DataFrame({
        "DataHora": data["hourly"]["time"],
        "Pressao_atmosferica(AUT)(mB)": data["hourly"]["surface_pressure"]
    })
    df_hourly["Data_Medicao"] = pd.to_datetime(df_hourly["DataHora"]).dt.date
    df_hourly["Pressao_atmosferica(AUT)(mB)"] = pd.to_numeric(df_hourly["Pressao_atmosferica(AUT)(mB)"], errors='coerce')

    # Média diária da pressão
    df_pressao = df_hourly.groupby("Data_Medicao")[["Pressao_atmosferica(AUT)(mB)"]].mean().reset_index()
    df_pressao["Data_Medicao"] = pd.to_datetime(df_pressao["Data_Medicao"])
    df_pressao["Pressao_atmosferica(AUT)(mB)"] = df_pressao["Pressao_atmosferica(AUT)(mB)"].round(1)

    # Junta tudo
    df_daily["Data_Medicao"] = pd.to_datetime(df_daily["Data_Medicao"])
    df_final = pd.merge(df_daily, df_pressao, on="Data_Medicao", how="left")

    # Renomear pressão
    df_final.rename(columns={
        "Pressao_atmosferica(AUT)(mB)": "Pressao_atmosferica_media_diario(AUT)(mB)"
    }, inplace=True)

    # Reorganiza as colunas na mesma ordem do dataset originaL
    colunas_ordenadas = [
        "Data_Medicao",
        "Precipitacao_total_diario(AUT)(mm)",
        "Pressao_atmosferica_media_diario(AUT)(mB)",
        "Temperatura_maxima_diario(AUT)(°C)",
        "Temperatura_minima_diario(AUT)(°C)",
        "Umidade_relativa_do_ar_media_diario(AUT)(%)",
        "Vento_velocidade_media_diario(AUT)(m/s)",
    ]
    df_final = df_final[colunas_ordenadas]


In [6]:
# ENGENHARIA DE ATRIBUTOS COMPLETA APLICADA A df_final

# Chuva acumulada dos últimos 3 dias
df_final['Chuva_3dias(mm)'] = df_final['Precipitacao_total_diario(AUT)(mm)'].rolling(window=3).sum()

# Temperatura média dos últimos 3 dias
df_final['Temp_media_3dias(°C)'] = df_final[['Temperatura_maxima_diario(AUT)(°C)', 'Temperatura_minima_diario(AUT)(°C)']].mean(axis=1).rolling(window=3).mean()

# Variação diária de temperatura
df_final['Variacao_Temp(°C)'] = df_final['Temperatura_maxima_diario(AUT)(°C)'] - df_final['Temperatura_minima_diario(AUT)(°C)']

# Variações absolutas de pressão e umidade em relação ao dia anterior
df_final['Variacao_Pressao(mB)'] = df_final['Pressao_atmosferica_media_diario(AUT)(mB)'].diff().abs()
df_final['Variacao_Umidade(%)'] = df_final['Umidade_relativa_do_ar_media_diario(AUT)(%)'].diff().abs()

# Preencher valores ausentes nas colunas auxiliares com a média
colunas_auxiliares = [
    'Chuva_3dias(mm)',
    'Temp_media_3dias(°C)',
    'Variacao_Pressao(mB)',
    'Variacao_Umidade(%)'
]
df_final[colunas_auxiliares] = df_final[colunas_auxiliares].apply(lambda x: x.fillna(x.mean()))

# Arredondar para 1 casa decimal
colunas_para_arredondar = colunas_auxiliares + ['Variacao_Temp(°C)']
df_final[colunas_para_arredondar] = df_final[colunas_para_arredondar].round(1)


In [7]:
# APLICAÇÃO DO MODELO AOS DADOS IMPORTADOS

modelo = joblib.load("modelo_random_forest.pkl")
features_usadas = joblib.load("lista_features.pkl")

X_novos_dados = df_final[features_usadas]

# Aplica o modelo e adiciona a probabilidade prevista
df_final["Probabilidade_Desastre(%)"] = modelo.predict(X_novos_dados) * 100
df_final["Probabilidade_Desastre(%)"] = df_final["Probabilidade_Desastre(%)"].clip(lower=0, upper=100).round().astype(int)

TESTE